### Integration 

We want to merge all of our 26 samples into one AnnData object.

In [1]:
import scvi
import numpy as np
import scanpy as sc
import pandas as pd
from pathlib import Path

ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
ribo_genes = pd.read_table(ribo_url, skiprows=2, header=None)


def preprocess(csv_path: Path):
    # Since scapy expects data to be in cellXgene format,
    # we'll have to transpose our geneXcell csv obtained from GSE171524
    adata = sc.read_csv(csv_path).T
    adata_original = adata.copy()
    # We want to filter out genes that do not appear in more than 10 cells
    # Genes expressed in extremely few cells are more likely to be noise than
    # anything of biological siginificane; could be ambient RNA or seq artifacts
    sc.pp.filter_genes(adata, min_cells=10)
    # Next, we only want to keep the genes with significant
    # biological variance and remove genes that are uninformative
    sc.pp.highly_variable_genes(
        adata, n_top_genes=2000, subset=True, flavor="seurat_v3"
    )
    # We only want to remove doublets
    # We take a probabilistic approach for this
    # And all previous filtering was done to prepare for this step
    # since this is computationally expensive and we'd like to eliminate unncessary compute
    # Read: https://www.sciencedirect.com/science/article/pii/S2405471220301952
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df["prediction"] = solo.predict(soft=False)
    doublets = df[df.prediction == "doublet"]

    adata_original.obs["Sample"] = str(csv_path).split("_")[
        1
    ]  #'data/GSM5226574_C51ctr_raw_counts.csv'
    adata_original.obs["doublet"] = adata_original.obs.index.isin(doublets.index)
    # We finally remove the doublets from our original matrix
    adata_original = adata_original[~adata_original.obs.doublet]

    # As a next step of filtering, we'd like to get
    # rid of cells that express fewer than 200 genes
    sc.pp.filter_cells(adata_original, min_genes=200)
    # also get rid of genes that appear
    # in fewer than 3 cells; this cuts out possible noise
    sc.pp.filter_genes(adata_original, min_cells=3)
    # Next, we mark mito genes
    adata_original.var["mt"] = adata_original.var_names.str.startswith("MT-")
    # and the ribo genes
    adata_original.var["ribo"] = adata_original.var_names.isin(ribo_genes[0].values)
    # we calculate our quality control matrix with helpful metrics
    sc.pp.calculate_qc_metrics(
        adata_original,
        qc_vars=["mt", "ribo"],
        percent_top=None,
        log1p=False,
        inplace=True,
    )
    # We only want to look into the 98%ile
    # of cells by gene count so only the cells where
    # a lot of genes are detected; cuts out empty reads from droplets
    upper_lim = np.quantile(adata_original.obs.n_genes_by_counts.values, 0.98)
    adata_original = adata_original[adata_original.obs.n_genes_by_counts < upper_lim]
    # Further we filter down to cells that show less than 20% mitochondrial genes
    adata_original = adata_original[adata_original.obs.pct_counts_mt < 20]
    # and less than 2% ribosomal genes
    adata_original = adata_original[adata_original.obs.pct_counts_ribo < 2]
    return adata_original


/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
samples = []
for csvs in Path("./data").iterdir():
    if csvs.is_file():
        samples.append(preprocess(csvs))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
You are using a CUDA device ('NVIDIA RTX PRO 6000 Blackwell Max-Q Workstation Edition') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aniruddham/c

Epoch 400/400: 100%|██████████| 400/400 [02:08<00:00,  3.90it/s, v_num=1, train_loss=473]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [02:08<00:00,  3.12it/s, v_num=1, train_loss=473]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lig

...

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [01:07<00:00,  5.90it/s, v_num=1, train_loss=320]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/lig

Epoch 148/400:  37%|███▋      | 148/400 [00:22<00:39,  6.45it/s, v_num=1, train_loss_step=0.434, train_loss_epoch=0.311]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.320. Signaling Trainer to stop.


/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/torch/utils/_contextlib.py:124: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabilities. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/torch/utils/_contextlib.py:124: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabilities. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)
/home/aniruddham/code/covid-lung-atlas/.venv/lib/python3.13/site-packages/scanpy/preprocessing/_simple.py:161: ImplicitModificationWarning: Trying to modify attribute `.obs` of

In [ ]:
adata = sc.concat(samples)

AnnData object with n_obs × n_vars = 85361 × 12892
    obs: 'Sample', 'doublet', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo'

In [6]:
from scipy.sparse import csr_matrix

adata.X = csr_matrix(adata.X)
adata.X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 59522170 stored elements and shape (85361, 12892)>

In [7]:
adata.write_h5ad("combined.h5ad")